# Model Training
Using transfer learning from VGG-16

[Keras Sample 1](https://medium.com/@14prakash/transfer-learning-using-keras-d804b2e04ef8)

[Keras Sample 2](https://www.kaggle.com/venuraja79/using-transfer-learning-with-keras)

In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Dense, GlobalAveragePooling2D, Flatten
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

Using TensorFlow backend.


In [2]:
img_width, img_height = 224, 224
model = applications.VGG16(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

In [3]:
# freeze layers
for layer in model.layers:
    layer.trainable = False
    
# add custom layers
x = model.output
x = GlobalAveragePooling2D()(x)
#x = Dense(128, activation="relu")(x)
#x = Dropout(0.5)(x)
x = Dense(256, activation="relu")(x)
prediction = Dense(2, activation="softmax", name="prediction")(x)

# generate new model
m = Model(inputs = model.input, outputs = prediction)
m.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

# Data Loader
Sample [data loader](https://gist.github.com/fchollet/0830affa1f7f19fd47b06d4cf89ed44d)

In [4]:
train_data_dir = 'data/catsndogs/train'
validation_data_dir = 'data/catsndogs/validation'
nb_train_samples = 855
nb_validation_samples = 95
epochs = 10
batch_size = 24

train = ImageDataGenerator(rescale = 1./255,
            horizontal_flip = True,
            fill_mode = "nearest",
            zoom_range = 0.3,
            width_shift_range = 0.3,
            height_shift_range=0.3,
            rotation_range=30)

train_generator = train.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

test  = ImageDataGenerator(rescale = 1./255)

test_generator = test.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 855 images belonging to 2 classes.
Found 95 images belonging to 2 classes.


In [6]:
dir(train_generator)
train_generator.samples

855

In [5]:
m.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=nb_validation_samples // batch_size)

m.save('latest.h5')

Epoch 1/10
35/35 [==============================] - 63s 2s/step - loss: 0.6237 - acc: 0.6510 - val_loss: 0.5186 - val_acc: 0.6528
Epoch 2/10
35/35 [==============================] - 61s 2s/step - loss: 0.4852 - acc: 0.7757 - val_loss: 0.3312 - val_acc: 0.9167
Epoch 3/10
35/35 [==============================] - 57s 2s/step - loss: 0.4308 - acc: 0.8118 - val_loss: 0.2748 - val_acc: 0.9167
Epoch 4/10
35/35 [==============================] - 59s 2s/step - loss: 0.3908 - acc: 0.8241 - val_loss: 0.2667 - val_acc: 0.9583
Epoch 5/10
35/35 [==============================] - 61s 2s/step - loss: 0.3633 - acc: 0.8486 - val_loss: 0.3703 - val_acc: 0.8472
Epoch 6/10
35/35 [==============================] - 60s 2s/step - loss: 0.3443 - acc: 0.8607 - val_loss: 0.2439 - val_acc: 0.9167
Epoch 7/10
35/35 [==============================] - 56s 2s/step - loss: 0.3448 - acc: 0.8427 - val_loss: 0.2347 - val_acc: 0.9583
Epoch 8/10
35/35 [==============================] - 58s 2s/step - loss: 0.3245 - acc: 0.85